In [2]:
%env CUDA_VISIBLE_DEVICES=0
from es_map import qdax_task

import numpy as np
import brax
from brax import jumpy as jp
from brax.envs import env
import jax.numpy as jnp
import jax

import matplotlib.pyplot as plt


from brax.envs import wrappers
from brax.io import html
from brax import envs

env: CUDA_VISIBLE_DEVICES=0


In [3]:
from es_map import behavior_map
from es_map import map_elite_utils
from es_map import jax_evaluate

In [4]:
key = jax.random.PRNGKey(777)
key_envs = jax.random.split(key, 1)

In [ ]:
run_path = "/scratch/ak1774/runs/large_files_jax/run-20220303_085708-pk4loduh"
import json
with open(run_path+'/config.json') as f:
    config = json.load(f)

In [9]:
run_id = "mfnlsrqm"
def id_to_path(id):
    from glob import glob    
    path = glob("/scratch/ak1774/runs/large_files_jax/*"+id, recursive = False)[0]
    return path
run_path = id_to_path(run_id)

import json
with open(run_path+'/config.json') as f:
    config = json.load(f)
theta = np.load(run_path+"/theta.npy")
from es_map import jax_evaluate


In [11]:
def create_env(env_name):
    if env_name == "ant":
        from es_map.qdax_envs.unidirectional_envs import ant, walker, hopper, halfcheetah, humanoid
        env = ant.QDUniAnt()
    elif env_name == "walker":
        from es_map.qdax_envs.unidirectional_envs import ant, walker, hopper, halfcheetah, humanoid
        env = walker.QDUniWalker()
    elif env_name == "hopper":
        from es_map.qdax_envs.unidirectional_envs import ant, walker, hopper, halfcheetah, humanoid
        env = hopper.QDUniHopper()
    elif env_name == "halfcheetah":
        from es_map.qdax_envs.unidirectional_envs import ant, walker, hopper, halfcheetah, humanoid
        env = halfcheetah.QDUniHalfcheetah()
    elif env_name == "humanoid":
        from es_map.qdax_envs.unidirectional_envs import ant, walker, hopper, halfcheetah, humanoid
        env = humanoid.QDUniHumanoid()
    elif env_name == "ant_omni":
        from es_map.qdax_envs.omnidirectional_envs import ant
        env = ant.QDOmniAnt()
    elif env_name == "humanoid_omni":
        from es_map.qdax_envs.omnidirectional_envs import humanoid
        env = humanoid.QDOmniHumanoid()
    else:
        raise "unknown env name"
        
    env = wrappers.EpisodeWrapper(env, episode_length=1000, action_repeat=1)
    env = wrappers.VectorWrapper(env, batch_size=10) # for each iteration, we evaluate the children and the parent performance in parallel
    env = wrappers.AutoResetWrapper(env) # not sure if this is nessasary, we only look at the first episode
    env = wrappers.VectorGymWrapper(env) 
    return env

In [13]:
env = create_env(config["env_name"])

In [17]:
action = env.action_space.sample()
obs = env.reset()

In [19]:
obs.shape

(10, 87)

In [ ]:
env.